In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [1]:
import re
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
# FILE_PATH = "/content/drive/MyDrive/Colab Notebooks/Arxiv Topic Classification/"
FILE_PATH = ""

## Convert deduplicated text into training data for preprocessing

In [29]:
deduplicated_arxiv = pd.read_csv("deduplicated_arxiv.csv")

In [30]:
deduplicated_arxiv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2276449 entries, 0 to 2276448
Data columns (total 5 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   title       object
 1   authors     object
 2   date        object
 3   abstract    object
 4   categories  object
dtypes: object(5)
memory usage: 86.8+ MB


In [31]:
deduplicated_arxiv.head()

,title,authors,date,abstract,categories
0,Calculation of prompt diphoton production cros...,"C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-...",2008-11-26,A fully differential calculation in perturba...,hep-ph
1,Sparsity-certifying Graph Decompositions,Ileana Streinu and Louis Theran,2008-12-13,"We describe a new algorithm, the $(k,\ell)$-...",math.CO cs.CG
2,The evolution of the Earth-Moon system based o...,Hongjun Pan,2008-01-13,The evolution of Earth-Moon system is descri...,physics.gen-ph
3,A determinant of Stirling cycle numbers counts...,David Callan,2007-05-23,We show that a determinant of Stirling cycle...,math.CO
4,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,Wael Abu-Shammala and Alberto Torchinsky,2013-10-15,In this paper we show how to compute the $\L...,math.CA math.FA


In [ ]:
# convert the categories into tuple of categories
df["categories"] = df["categories"].apply(lambda categories : tuple(categories.split()))

df["num_categories"] = df["categories"].apply(lambda x : len(x))

In [ ]:
df.head()

In [ ]:
df.categories.nunique()

In [ ]:
categories = df["categories"].tolist()

# Getting all unique categories by flattening the 'categories' column
# and creating a set out of the resultant list.
unique_categories = {}
for row in categories:
    for category in row:
        unique_categories[category] = unique_categories.get(category, 0) + 1

print(f"Num. unique categories: {len(unique_categories)}")

In [ ]:
print(df.abstract[0])

In [ ]:
# Since we are going to use only abstract and title, and categories for training, we are going to concatenate title and abstract and drop other columns
NUM = 1000
text = df.title[NUM] + df.abstract[NUM]
print(df.title[NUM])
print()
print(df.abstract[NUM])
print()
print(text)

In [ ]:
df["text"] = df.title + df.abstract

In [ ]:
df.head()

In [ ]:
df.drop(["title", "abstract", "authors", "date", "num_categories"], axis = 1, inplace = True)

In [ ]:
df.head()

In [ ]:
df.to_csv(FILE_PATH + "training_data_for_preprocessing.csv")

## Preprocessing

In [4]:
df = pd.read_csv(FILE_PATH + "training_data_for_preprocessing.csv")

In [5]:
df.head()

,Unnamed: 0,categories,text
0,0,hep-ph,Calculation of prompt diphoton production cros...
1,1,math.CO cs.CG,Sparsity-certifying Graph Decompositions We d...
2,2,physics.gen-ph,The evolution of the Earth-Moon system based o...
3,3,math.CO,A determinant of Stirling cycle numbers counts...
4,4,math.CA math.FA,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...


In [6]:
df.drop(["Unnamed: 0"], axis = 1, inplace = True)

In [7]:
df.head()

,categories,text
0,hep-ph,Calculation of prompt diphoton production cros...
1,math.CO cs.CG,Sparsity-certifying Graph Decompositions We d...
2,physics.gen-ph,The evolution of the Earth-Moon system based o...
3,math.CO,A determinant of Stirling cycle numbers counts...
4,math.CA math.FA,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...


In [15]:
print(df.text[1])

sparsitycertifi graph decomposit describ new algorithm kellpebbl game color use obtain character famili kellspars graph algorithm solut famili problem concern tree decomposit graph special instanc spars graph appear rigid theori receiv increas attent recent year particular color pebbl gener strengthen previou result lee streinu give new proof tuttenashwilliam character arbor also present new decomposit certifi sparsiti base kellpebbl game color work also expos connect pebbl game algorithm previou spars graph algorithm gabow gabow westermann hendrickson


In [22]:
import nltk
nltk.download("stopwords")
from nltk.stem import PorterStemmer
ps = PorterStemmer()

from nltk.corpus import stopwords
stop_word_collection = stopwords.words('english')

import string

from tqdm import tqdm

def text_preprocess(text, progress_bar):
    # Update the progress bar
    progress_bar.update(1)

    # Replace all punctuation with white space
    translator = str.maketrans(string.punctuation, ' ' * len(string.punctuation))
    text = text.translate(translator)

    # Remove all numbers and words containing numbers
    text = re.sub(r'\w*\d\w*', ' ', text).strip()

    # Changes to lower case
    text = text.lower()

    # Remove all stop words
    text = ' '. join(word for word in text.split() if word not in stop_word_collection)

    # Stemming of all words
    text = [ps.stem(word) for word in text.split()]
    text = ' '.join(text)
    return text

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/nirajan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [63]:
df.text[900_000]

'Degeneration of globally hyperbolic maximal anti-de Sitter structures\n  along rays  Using the parameterisation of the deformation space of GHMC anti-de Sitter\nstructures on $S \\times \\mathbb{R}$ by the cotangent bundle of the\nTeichm\\"uller space of $S$, we study how some geometric quantities, such as the\nLorentzian Hausdorff dimension of the limit set, the width of the convex core\nand the H\\"older exponent, degenerate along rays of quadratic differentials.\n'

In [64]:
progress_bar = tqdm(total=100_000)

NUM = 9
df.text[NUM*100_000: NUM*100_000 + 100_000] = df.text[NUM*100_000: NUM*100_000 + 100_000] \
                                                .apply(text_preprocess, args=(progress_bar,))
# Close the progress bar
progress_bar.close()

100%|██████████████████████████████████| 100000/100000 [03:19<00:00, 502.47it/s]


In [65]:
print(df.text[900_000])

degener global hyperbol maxim anti de sitter structur along ray use parameteris deform space ghmc anti de sitter structur time mathbb r cotang bundl teichm uller space studi geometr quantiti lorentzian hausdorff dimens limit set width convex core h older expon degener along ray quadrat differenti


In [66]:
df.to_csv("preprocessed_csv.csv", index = False)

## First 1 million data

In [3]:
data = pd.read_csv("preprocessed_csv.csv")

In [4]:
data.head()

,categories,text
0,hep-ph,calcul prompt diphoton product cross section t...
1,math.CO cs.CG,sparsitycertifi graph decomposit describ new a...
2,physics.gen-ph,evolut earthmoon system base dark matter field...
3,math.CO,determin stirl cycl number count unlabel acycl...
4,math.CA math.FA,dyadic lambdaalpha lambdaalpha paper show comp...


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2276449 entries, 0 to 2276448
Data columns (total 2 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   categories  object
 1   text        object
dtypes: object(2)
memory usage: 34.7+ MB


In [6]:
data.iloc[999_999].text

'proactiv intervent downtrend employe attrit use artifici intellig techniqu predict employe attrit beforehand enabl manag take individu prevent action use ensembl classif model techniqu linear regress model could predict accur employe predict lead time separ individu reason caus attrit prior intim employe attrit enabl manag take prevent action retain employe manag busi consequ attrit deploy model help downtrend employe attrit help manag manag team effect model cover natur calam unforeseen event occur individu level like accid death etc'

In [7]:
data.iloc[1_000_000].text

'Random motion on finite rings, II: Noncommutative rings  We extend our previous study of Markov chains on finite commutative rings\n(arXiv:1605.05089) to arbitrary finite rings with identity. At each step, we\neither add or multiply by a randomly chosen element of the ring, where the\naddition (resp. multiplication) distribution is uniform (resp. conjugacy\ninvariant). We prove explicit formulas for some of the eigenvalues of the\ntransition matrix and give lower bounds on their multiplicities. We also give\nrecursive formulas for the stationary distribution and prove that the mixing\ntime is bounded by an absolute constant. For the matrix rings $M_2(\\mathbb\nF_q),$ we compute the entire spectrum explicitly using the representation\ntheory of $\\text{GL}_2(\\mathbb F_q),$ as well as the stationary probabilities.\n'

In [8]:
sample_data = data.iloc[0:999_999]

In [9]:
sample_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999999 entries, 0 to 999998
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   categories  999999 non-null  object
 1   text        999999 non-null  object
dtypes: object(2)
memory usage: 15.3+ MB


In [10]:
sample_data.text

0         calcul prompt diphoton product cross section t...
1         sparsitycertifi graph decomposit describ new a...
2         evolut earthmoon system base dark matter field...
3         determin stirl cycl number count unlabel acycl...
4         dyadic lambdaalpha lambdaalpha paper show comp...
                                ...                        
999994    dynam coupl dilut magnet impur quantum spin li...
999995    recognis cardiac abnorm wearabl devic photople...
999996    evolut skyrmion crystal fe co si like quasi tw...
999997    benedick amrein berthier type theorem relat tw...
999998    constraint scalar tensor model gauss bonnet co...
Name: text, Length: 999999, dtype: object

In [11]:
sample_data.to_csv("first 1 million.csv", index = False)